In [3]:
%%writefile modularize/get_data.py
import os
import requests
import zipfile
from pathlib import Path

#setup path to data folder
data_path = Path('data/')
image_path = data_path/'pizza_steak_sushi'

#if the iumage folder doesn't exist, download and prepare the data
if image_path.is_dir():
    print(f'{image_path} already exists')
else:
    print(f'{image_path} does not exist, downloading files...')
    image_path.mkdir(parents=True, exist_ok=True)

#download the data to the image_path
with open('data/pizza_steak_sushi.zip', 'wb') as f:
    request = requests.get("https://github.com/mrdbourke/pytorch-deep-learning/raw/main/data/pizza_steak_sushi.zip")
    print("Downloading pizza, steak, sushi data...")
    f.write(request.content)

#unzip the data
with zipfile.ZipFile('data/pizza_steak_sushi.zip', 'r') as zip_ref:
    zip_ref.extractall(image_path)
    print("Data unzipped!")

#remove the zip file
os.remove(data_path/'pizza_steak_sushi.zip')


Writing modularize/get_data.py


In [11]:
%%writefile modularize/data_setup.py
"""
Contains functionality for creating PyTorch DataLoaders for 
image classification data.
"""
import os

from torchvision import datasets, transforms
from torch.utils.data import DataLoader

NUM_WORKERS = os.cpu_count()

def create_dataloaders(
    train_dir: str, 
    test_dir: str, 
    transform: transforms.Compose, 
    batch_size: int, 
    num_workers: int=NUM_WORKERS
):
  """Creates training and testing DataLoaders.

  Takes in a training directory and testing directory path and turns
  them into PyTorch Datasets and then into PyTorch DataLoaders.

  Args:
    train_dir: Path to training directory.
    test_dir: Path to testing directory.
    transform: torchvision transforms to perform on training and testing data.
    batch_size: Number of samples per batch in each of the DataLoaders.
    num_workers: An integer for number of workers per DataLoader.

  Returns:
    A tuple of (train_dataloader, test_dataloader, class_names).
    Where class_names is a list of the target classes.
    Example usage:
      train_dataloader, test_dataloader, class_names = \
        = create_dataloaders(train_dir=path/to/train_dir,
                            test_dir=path/to/test_dir,
                            transform=some_transform,
                            batch_size=32,
                            num_workers=4)
  """
  # Use ImageFolder to create dataset(s)
  train_data = datasets.ImageFolder(train_dir, transform=transform)
  test_data = datasets.ImageFolder(test_dir, transform=transform)

  # Get class names
  class_names = train_data.classes

  # Turn images into data loaders
  train_dataloader = DataLoader(
      train_data,
      batch_size=batch_size,
      shuffle=True,
      num_workers=num_workers,
      pin_memory=True,
  )
  test_dataloader = DataLoader(
      test_data,
      batch_size=batch_size,
      shuffle=True,
      num_workers=num_workers,
      pin_memory=True,
  )

  return train_dataloader, test_dataloader, class_names

Writing modularize/data_setup.py


In [12]:
%%writefile modularize/model_builder.py
"""
Contains PyTorch model code to instantiate a TinyVGG model.
"""
import torch
from torch import nn 

class TinyVGG(nn.Module):
    """Creates the TinyVGG architecture.    
    Replicates the TinyVGG architecture from the CNN explainer website in PyTorch.
    See the original architecture here: https://poloclub.github.io/cnn-explainer/   
    Args:
        input_shape: An integer indicating number of input channels.
        hidden_units: An integer indicating number of hidden units between layers.
        output_shape: An integer indicating number of output units.
    """
    def __init__(self, input_shape: int, hidden_units: int, output_shape: int) -> None:
        super().__init__()
        self.conv_block_1 = nn.Sequential(
            nn.Conv2d(in_channels=input_shape, 
                        out_channels=hidden_units, 
                        kernel_size=3, 
                        stride=1, 
                        padding=0),  
            nn.ReLU(),
            nn.Conv2d(in_channels=hidden_units, 
                        out_channels=hidden_units,
                        kernel_size=3,
                        stride=1,
                        padding=0),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2,
                            stride=2)
        )
        self.conv_block_2 = nn.Sequential(
            nn.Conv2d(hidden_units, hidden_units, kernel_size=3, padding=0),
            nn.ReLU(),
            nn.Conv2d(hidden_units, hidden_units, kernel_size=3, padding=0),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        self.classifier = nn.Sequential(
            nn.Flatten(),
            #each layer of our network compresses and changes the shape of our inputs data.
            nn.Linear(in_features=hidden_units*13*13,
                        out_features=output_shape)
        )

    def forward(self, x: torch.Tensor):
        x = self.conv_block_1(x)
        x = self.conv_block_2(x)
        x = self.classifier(x)
        return x
        #return self.classifier(self.conv_block_2(self.conv_block_1(x))) # <- leverage the benefits of operator fusion

Writing modularize/model_builder.py


In [5]:
%%writefile modularize/engine.py
"""
Contains functionality for training and testing a pytorch model
"""

import torch

from tqdm.auto import tqdm
from typing import Dict, List, Tuple

def train_step(
    model: torch.nn.Module,
    dataloader: torch.utils.data.DataLoader,
    loss_fn: torch.nn.Module,
    optimizer: torch.optim.Optimizer,
    device: torch.device,
) -> Tuple[float, float]:
    """trains a model for a single epoch

    turns a target model to training mode and then runs through
    all of the required training steps (forward pass, loss calculation, optimizer step)

    Args:
        model: A PyTorch model to train.
        dataloader: A PyTorch DataLoader with training data.
        loss_fn: A PyTorch loss function.
        optimizer: A PyTorch optimizer.
        device: A PyTorch device.


    returns:
        A tuple of (loss, accuracy) for the epoch.
        in the form (train_loss, train_accuracy) for example:
        (1.2345, 0.5678)
    """
    #set model to training mode
    model.train()

    #set train loss and train accuracy to 0
    train_loss = 0, train_acc = 0

    #loop through data loader data batches
    for batch, (X,y) in enumerate(dataloader):
        #send data to device
        X,y = X.to(device), y.to(device)
        #forward pass
        preds = model(X)
        #calculate loss
        loss = loss_fn(preds, y)
        train_loss += loss.item()
        #optimzer zero grad
        optimizer.zero_grad()
        #backward pass
        loss.backward()
        #optimizer step
        optimizer.step()
        #calculate accuracy
        pred_class = torch.argmax(sofmax(preds, dim=1), dim=1)
        train_acc += (pred_class == y).sum().item() / len(preds)

    #adjust metrics to get average loss and accuracy per batch
    train_loss = train_loss / len(dataloader)
    train_acc = train_acc / len(dataloader)
    return train_loss, train_acc


def test_step(
    model: torch.nn.Module,
    dataloader: torch.utils.data.DataLoader,
    loss_fn: torch.nn.Module,
    device: torch.device,
) -> Tuple[float, float]:
    """tests a model for a single epoch

    Turns a target model to evaluation mode and then runs through
    a forward pass on a testing dataset

    Args:
        model: A PyTorch model to test.
        dataloader: A PyTorch DataLoader with testing data.
        loss_fn: A PyTorch loss function.
        device: A PyTorch device.

    returns:
        A tuple of (loss, accuracy) for the epoch.
        in the form (test_loss, test_accuracy) for example:
        (1.2345, 0.5678)
    """
    #set model to evaluation mode
    model.eval()

    #set test loss and test accuracy to 0
    test_loss = 0, test_acc = 0

    #turn on inference context manager
    with torch.inference_mode():
        #loop through data loader data batches
        for batch, (X,y) in enumerate(dataloader):
            #send data to device
            X,y = X.to(device), y.to(device)
            #forward pass
            preds = model(X)
            #calculate loss
            loss = loss_fn(preds, y)
            test_loss += loss.item()
            #calculate accuracy
            pred_class = torch.argmax(sofmax(preds, dim=1), dim=1)
            test_acc += (pred_class == y).sum().item() / len(preds)
            


Overwriting modularize/engine.py


In [1]:
%%writefile modularize/utils.py
"""
contains a bunch a utility functions for training and saving
"""
import torch
from pathlib import Path
def save_model(model: torch.nn.Module, target_dir: str, model_name: str):
    """saves a model to a target directory
    Args:
        model: a pytorch model to save
        target_dir: a directory for saving the model to
        model_name: a filename for the saved model sould include either .pth or .pt as extension.
    
    example:
        save_model(model_0, models, tinyvgg_model.pth)
    """
    #create target directory
    target_dir_path = Path(target_dir)
    target_dir_path.mkdir(True, True)

    #create model save path
    assert model_name.endswith(".pth") or model_name.endswith(".pt"), "wrong extension, pth or pt pls"
    model_save_path = target_dir_path / model_name

    #save state dict
    print(f'[INFO] saving model to: {model_save_path}')
    torch.save(model.state_dict, model_save_path)


Writing modularize/utils.py


In [2]:
%%writefile modularize/train.py
"""
trains a pytorch image classification model using device-agnostic code.
"""

import os
import torch
import data_setup, engine, model_builder, utils

from torchvision import transforms

# Setup hyperparameters
NUM_EPOCHS = 5
BATCH_SIZE = 32
HIDDEN_UNITS = 10
LEARNING_RATE = 0.001

# Setup directories
train_dir = "data/pizza_steak_sushi/train"
test_dir = "data/pizza_steak_sushi/test"

# Setup target device
device = "cuda" if torch.cuda.is_available() else "cpu"

# Create transforms
data_transform = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.ToTensor()
])

# Create DataLoaders with help from data_setup.py
train_dataloader, test_dataloader, class_names = data_setup.create_dataloaders(
    train_dir=train_dir,
    test_dir=test_dir,
    transform=data_transform,
    batch_size=BATCH_SIZE
)

# Create model with help from model_builder.py
model = model_builder.TinyVGG(
    input_shape=3,
    hidden_units=HIDDEN_UNITS,
    output_shape=len(class_names)
).to(device)

# Set loss and optimizer
loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),
                            lr=LEARNING_RATE)

# Start training with help from engine.py
engine.train(model=model,
            train_dataloader=train_dataloader,
            test_dataloader=test_dataloader,
            loss_fn=loss_fn,
            optimizer=optimizer,
            epochs=NUM_EPOCHS,
            device=device)

# Save the model with help from utils.py
utils.save_model(model=model,
                target_dir="models",
                model_name="tinyvgg_model.pth")

Writing modularize/train.py
